Подключим нужные для работы библиотеки

In [134]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import mean_squared_error, roc_auc_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.pipeline import Pipeline

Для задачи прогнозировнаия времени доставки по заказам выбираем следующие столбцы:
через запрос к БД:
select o.customer_id, c.address, o.order_id, o.order_dt,  d.delivery_dt, o.payment_method, op.product_id, op.amount
from orders as o
inner join delivery_info as d
on o.order_id = d.order_id
inner join customers as c
on o.customer_id = c.customer_id
inner join orders_product as op
on o.order_id = op.order_id

Полученные поля:
customer_id,address,order_id,order_dt,delivery_dt,payment_method,product_id,amount
                ID Клиента [customer_id]
                Город доставки  [address]
                ID заказа [order_id]
                Дата заказа [order_dt]
                Дата поставки [delivery_dt]
                Тип оплаты [payment_method]
                ID продукта [product_id]
                Количество продукта в заказе [amount]
Выбор стобцов обусловлен влиянием, которые значения могут оказать на целевую переменную (время доставки в днях).

 Целевая переменная - это произвольное положительное число, Тип задачи - Линейная Регрессия.

 Для оценки качества модели прогнозирования будем использовать Среднеквадратичную ошибку (как наиболее корректную метрику для методов линейной регрессии)


Для использования метода прогнозирования, Преобразуенм данные:
 -  Для каждого заказа получить прогнозируемую переменную — создаенм вычисляемое поле =  время доставки в днях (разница в днях между датой заказа и датой доставки =  days_deliver, затем удаляем данные поля (order_dt) и (delivery_dt)
 - перекодируем адреса через ohe в числовые поля
 - метод платежа также преобразуем в число

In [118]:
df = pd.read_csv('hw_14_6_sql.csv')
df['order_dt'] = pd.to_datetime(df['order_dt'])
df['delivery_dt'] = pd.to_datetime(df['delivery_dt'])
df['days_deliver'] = (df['delivery_dt'] - df['order_dt']).dt.days
df.drop(['delivery_dt', 'order_dt'], axis= 1 , inplace= True )
df['payment_method'] = df['payment_method'].apply(lambda x: 1 if x == 'online' else 1)
oe_style = OneHotEncoder()
oe_results = oe_style.fit_transform(df[["address"]])
oe_results.toarray()
pd.DataFrame(oe_results.toarray(), columns=oe_style.categories_).head()
df = df.join(pd.DataFrame(oe_results.toarray(), columns=oe_style.categories_[0]))
df.drop('address', axis= 1 , inplace= True )

Разбиваем данные на целевые и признаки, для анализа с помощью методов линейной регрессии

In [121]:
y = df['days_deliver']
X = df.loc[:, df.columns != 'days_deliver']
X = X.apply(pd.to_numeric)
sc = StandardScaler()
X_sc = sc.fit_transform(X)

In [162]:
lr = LinearRegression()
lr.fit(X_sc,y)
y_pred = lr.predict(X_sc)
mean_squared_error(y, y_pred, squared=False)

16.139001783690986

Оцениваем эффективность работы модели - через среднеквадратичную ошибку.
В первом расчете метод линейной регрессии дает неплохой результат

In [163]:
pl = Pipeline([('sc', sc), ('model', lr)])
y_pred = cross_val_predict(pl, X_sc, y)
mean_squared_error(y_pred, y, squared=False)

64256102865749.57

Но после проверки с помощью Кросс-валидации оказывается, что метод переобучился и не дает нужного стабильного результата

In [164]:
dt_r = DecisionTreeRegressor(max_depth=5)
dt_r.fit(X_sc,y)
y_pred = dt_r.predict(X_sc)
mean_squared_error(y_pred, y, squared=False)

17.206691077362382

In [165]:
y_pred = cross_val_predict(dt_r, X_sc, y)
mean_squared_error(y_pred, y, squared=False)

17.317983985073607

Модель дерева решения дает результат немного хуже при ограничении глубины на кросс-валмдлации

In [159]:
rf_r = RandomForestRegressor()
rf_r.fit(X_sc,y)
y_pred = rf_r.predict(X_sc)
mean_squared_error(y_pred, y, squared=False)

11.142484389385027

In [160]:
y_pred = cross_val_predict(rf_r, X_sc, y)
mean_squared_error(y_pred, y, squared=False)

14.864124644644077

Метод случаного леса дает наилучший результат на кросс валидации.

## Вывод:
Алгоритм случайного лесадает наилучший результат (метрика:
Среднеквадратичная ошибка)